In [ ]:
# Date: 13.02.25
# Purpose: Create a system users can access when a customer contacts them for a rate negotiation.
# Theme: Agent
# Status: bu

In [ ]:
# Design notes:
# - Data: credit policies, competitor insights, past rate offers and reactions
# - Build: No CI/CD/CT atm, just file ingestion and set up. Thinking 
# - - Agent enters cust no triggering SQL query to obtain key cust data (loan amt, tenure, recent engagements, etc.)
# - - Cust data is integrated into a system prompt & run against credit policy to assess if any room for discount (if applicable)
# - - (does this add value?) Query run against vector db finding similar cust who were offered & accepted deal (cust ranked by smallest discount)
# - - Scan of any competitor data (to see if any insights that can aid this conversation/rate neg)


# Wait, to start i just want some SQlite db (cust data, product data, trans data). 
# I want to enter a cust number and be able to pull back info from each of those db


# - - Code can combine inputs from those fields, plus add a system prompt.
# - - Credit policies: need to use certain fields from input to get parameters from credit policies

# - - Three seperate vector DB searchs?
# - - Possibly 
# - - Combine results of the above searches and send to llm with system prompt 

In [ ]:
#!pip install google-generativeai
#pip install python-dotenv
#!pip install tabulate


In [ ]:
print('farts')

### Data (create & populate tbls)

In [ ]:
# drop tables (if required)
import sqlite3
# Connect to SQLite (or create the database file)
conn = sqlite3.connect("bank_data.db")
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS transactions;")
cursor.execute("DROP TABLE IF EXISTS products;")
cursor.execute("DROP TABLE IF EXISTS customers;")

In [ ]:
# -- 1.0 -- Create tables (and dummy data)

import sqlite3
# Connect to SQLite (or create the database file)
conn = sqlite3.connect("bank_data.db")
cursor = conn.cursor()

# Do not forget that as fields are added here, that also need to be Manually added to sys (as this is still in dev stage)


# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    location TEXT,
    tenure INTEGER,
    occupation TEXT,
    industry_employed TEXT,
    income_monthly INTEGER,
    credit_hist TEXT,
    defaults INTEGER                  
                        
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    customer_id INTEGER,
    product TEXT,
    description TEXT,
    rate REAL,
    balance INTEGER,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    customer_id INTEGER,
    date TEXT,
    transaction_description TEXT,
    transaction_classification TEXT,
    trasaction_amount REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
''')



# Credit hist: Excellent, Good, Fair, Poor, Bad
# Insert data into customers
data_customers = [
    (1, "Alice Johnson", 34, "Sydney", 10, "Stock broker", "Finance", 10000, "Fair", 0),
    (2, "Bob Smith", 45, "Melbourne", 15, "Waiter/Waitress", "Accommodation", 500, "Poor", 2),
    (3, "Charlie Brown", 29, "Brisbane", 5, "Architect", "Construction", 8000, "Good", 0), 
    (4, "Alice Johnson", 34, "Sydney", 3, "Engineer", "Mining", 12000, "Good", 0),  
    (5, "Bob Smith", 41, "Melbourne", 2, "Analyst", "Finance", 9000, "Excellent", 0),  
    (6, "Daisy Clark", 27, "Perth", 4, "Bartender", "Accommodation", 4000, "Bad", 4),   
    (7, "Ethan Wright", 36, "Adelaide", 1, "Teacher", "Education", 6000, "Good", 0),   
    (8, "Fiona Lewis", 30, "Canberra", 5, "Police", "Government", 3000, "Fair", 0),  
    (9, "George Miller", 28, "Gold Coast", 3, "Travel agent", "Travel", 3500, "Poor", 1),  
    (10, "Hannah Scott", 39, "Hobart", 4, "Florist", "Culture", 1500, "Fair", 0)
]
cursor.executemany("INSERT INTO customers VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", data_customers)

# Insert data into products
data_products = [
    (1, "Home Loan", "30-year fixed mortgage", 3.5, 250000),
    (2, "Credit Card", "Platinum rewards card", 18.0, 2000),
    (3, "Savings Account", "High-interest savings", 2.0, 16000),
    (3, "Auto Loan", "New car financing", 4.25, 6000),
    (3, "Personal Loan", "Unsecured personal loan", 7.99, 2300),
    (3, "Student Loan", "Federal student loan", 5.05, 18000),
    (2, "Business Loan", "Small business line of credit", 6.75, 180000),
    (2, "Home Equity Loan", "Fixed-rate home equity", 4.5, 240000),
    (1, "Checking Account", "Free checking with rewards", 0.1, 5500),
    (1, "Money Market Account", "High-yield money market", 1.75, 30000),
    (1, "Certificate of Deposit", "12-month CD", 2.25, 10000),
    (2, "Credit Card", "Cash back rewards card", 16.99, 900),
    (3, "Mortgage", "15-year fixed mortgage", 3.0, 525000),
    (1, "Investment Account", "Robo-advisor balanced portfolio", 7.5, 17900),
    (2, "Retirement Account", "Traditional IRA", 6.0, 620000),
    (3, "Credit Card", "Low-interest card", 12.99, 4000),
    (2, "Auto Loan", "Used car financing", 5.25, 600),
    (1, "Personal Line of Credit", "Secured line of credit", 8.5, 2000),
    (1, "Business Credit Card", "Travel rewards business card", 17.99, 3500),
    (2, "Savings Account", "Youth savings account", 1.5, 9000)
]
cursor.executemany("INSERT INTO products VALUES (?, ?, ?, ?, ?)", data_products)

# Insert data into transactions
data_transactions = [
    (1, "2025-01-15", "Loan Payment", "withdrawal", 2000.00),
    (2, "2025-01-20", "Credit Card Purchase - gambling", "withdrawal", 150.75),
    (3, "2025-01-22", "Deposit", "deposit", 500.00),
    (1, "2025-01-25", "Grocery Shopping", "withdrawal",87.50),
    (2, "2025-01-28", "Salary Deposit", "deposit", 3500.00),
    (3, "2025-01-30", "Utility Bill", "withdrawal", 120.00),
    (1, "2025-02-01", "Rent Payment", "withdrawal", 1500.00),
    (2, "2025-02-03", "Gas Station", "withdrawal", 45.00),
    (3, "2025-02-05", "Online Purchase - alchohol","withdrawal", 65.99),
    (1, "2025-02-07", "Restaurant Dinner", "withdrawal", 78.25),
    (1, "2025-02-10", "Mobile Phone Bill", "withdrawal", 85.00),
    (1, "2025-02-12", "Gym Membership", "withdrawal", 50.00),
    (1, "2025-02-15", "Loan Payment", "withdrawal", 2000.00),
    (1, "2025-02-18", "Clothing Store", "withdrawal", 129.99),
    (1, "2025-02-20", "Credit Card Payment", "withdrawal", 500.00),
    (1, "2025-02-22", "Freelance Income", "deposit", 750.00),
    (2, "2025-02-25", "Dentist Appointment", "withdrawal", 175.00),
    (2, "2025-02-28", "Salary Deposit", "deposit", 3500.00),
    (2, "2025-03-01", "Car Insurance", "withdrawal", 110.00),
    (2, "2025-03-03", "Supermarket", "withdrawal", 95.75),
    (2, "2025-03-05", "Coffee Shop", "withdrawal", 4.50),
    (2, "2025-03-07", "Book Purchase", "withdrawal", 29.99),
    (2, "2025-03-10", "Public Transport", "withdrawal", 40.00),
    (3, "2025-03-12", "Home Improvement Store", "withdrawal", 87.50),
    (3, "2025-03-15", "Loan Payment", "withdrawal", 2000.00),
    (3, "2025-03-18", "Movie Theater", "withdrawal", 25.00),
    (3, "2025-03-20", "Credit Card Purchase", "withdrawal", 199.99),
    (3, "2025-03-22", "Deposit", "deposit", 300.00),
    (3, "2025-03-25", "Hair Salon", "withdrawal", 65.00),
    (3, "2025-03-28", "Salary Deposit", "deposit", 3500.00)

]
cursor.executemany("INSERT INTO transactions VALUES (?, ?, ?, ?, ?)", data_transactions)

# Commit changes and close connection
conn.commit()
conn.close()

print("Database created and populated successfully.")


In [ ]:
# -- 2.0 Query db (verify code and data returned) --

import sqlite3
from tabulate import tabulate
# Connect to the database
conn = sqlite3.connect("bank_data.db")
cursor = conn.cursor()

# 1.0 Customer query
query = """
SELECT
    customer_id AS ID, 
    name AS CustomerName, 
    age AS Age, 
    location,
    tenure AS TenureYears,
    occupation,
    industry_employed,
    income_monthly,
    credit_hist,
    defaults   
FROM customers
ORDER BY customer_id ASC
LIMIT 3;
"""
 
# Execute the query
cursor.execute(query)
rows = cursor.fetchall()
# Print results
for row in rows:
    print(row)
# Close the connection
#conn.close()


print('\n','-'*5, 'new', '-'*5)
# 2.0 Product query
query2 = """
SELECT 
    customer_id AS ID, 
    product,
    description,
    rate,
    balance
FROM products
ORDER BY customer_id ASC
LIMIT 3;
"""
# Execute the query
cursor.execute(query2)
rows = cursor.fetchall()
# Print results
for row in rows:
    print(row)
# Close the connection
#conn.close()


print('\n','-'*5, 'new', '-'*5)
# 3.0 Transaction query
query3 = """
SELECT 
    customer_id AS ID, 
    date,
    transaction_description,
    transaction_classification,
    trasaction_amount
FROM transactions
ORDER BY customer_id ASC
LIMIT 3;
"""
# Execute the query
cursor.execute(query3)
rows = cursor.fetchall()
# Print results
for row in rows:
    print(row)
# Close the connection
#conn.close()


print('\n','-'*5, 'new', '-'*5)
# 4.0 Join query
query4 = """
SELECT 
    a.customer_id AS ID, 
    a.name AS CustomerName, 
    a.age AS Age, 
    a.tenure AS TenureYears,
    b.product,
    b.description
FROM customers a
LEFT JOIN products b
  ON a.customer_id = b.customer_id
WHERE a.customer_id = 4  
ORDER BY a.customer_id ASC
LIMIT 5;
"""
cursor.execute(query4)
rows = cursor.fetchall()
# Get column names
headers = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=headers, tablefmt="grid"))
data = tabulate(rows, headers=headers, tablefmt="grid")
conn.close()




### Initial set up

In [ ]:
# -- 3.0 Set up LLM --
import os
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
#from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')
import os
import google.generativeai as genai
genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 0.2,
  "top_p": 0.90,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)
chat_session = model.start_chat(
  history=[  ])
message = "this is a test only"
response = chat_session.send_message(message)
print(response.text)

In [ ]:
# -- 4.0 System prompts --
tbls = """There are 3 seperate tables, that are related by a common field 'customer_id'
         The tables are 
         Table 1: customers
         Columns: customer_id, name, age, location, tenure, occupation, industry_employed, income_monthly, credit_hist, defaults 
         Description: includes values such as income, occupation, credit history, defaults

         Table 2: products
         Columns: customer_id, product, description, rate, balance
         Description: includes values such as products, balances and rates

         Table 3: transactions
         Columns: customer_id, date, transaction_description, transaction_classification, trasaction_amount
         Description: transaction_classification column includes two values: deposits & withdrawals, 
                      transaction_description column includes many values, i.e. 
                      Credit Card Purchase - gambling, Deposit, Grocery Shopping, Salary Deposit, Utility Bill, Rent Payment, etc.
        """

sys = """You will create SQL code that can execute on a SQlite db.
         If request requires information contained in more that one table, 
         You have the ability to write subqueries, joins, etc. to get the data required.
         If the request requires aggregation, calculations, etc. you will include this in your query.
         Please pull back only fields, that you believe are relevant to the query.
         You will output only the code required to be execute on the SQLitedb.
         You will not include any descriptions, explainations, etc. - just output the code"""
         


In [ ]:
# # -- 5.0 LLM query, response and cleaning (format) --
# # Query (pre credit policy - so just checking can pull back data)
# query = "Can you return all data for customer 1?"

# prompt = sys + tbls + query
# response = chat_session.send_message(prompt)
# resp = response.text
# print(response.text)

# # Clean (format)
# # Clean
# resp1 = resp.replace('```sql','').replace('```','')
# #print(resp1)

In [ ]:
# # -- 6.0 Query db, using LLM cleaned (format) llm response --
# import sqlite3
# from tabulate import tabulate
# # Connect to the database
# conn = sqlite3.connect("bank_data.db")
# cursor = conn.cursor()

# print('\n','-'*5, 'new', '-'*5)
# # 4.0 Join query
# query5 = f"""{resp1}"""
# cursor.execute(query5)
# rows = cursor.fetchall()
# # Get column names
# headers = [desc[0] for desc in cursor.description]
# print(tabulate(rows, headers=headers, tablefmt="grid"))
# data = tabulate(rows, headers=headers, tablefmt="grid")
# conn.close()

__Credit policy:__<br>
Asked ChatGPT for this: can you create a one page credit policy for an australian financial institution. <br>
The intent of the policy is to provide rules and guidance to staff in the event a customer contacts our bank to seek a rate negotiation. <br>
There should be a number of categories (with subcategories) in the policy. <br>
These should be <br>
__Credit History__ (has client defaulted, do they spend funds at high risk places), <br>
__Borrower type__ (age, location, occupation, industry of employment), <br>
__Financial position__ (income and expenses) 

In [39]:
with open('data/Credit Policy.txt', "r", encoding="utf-8") as f:
    creditpol = f.read()
print(creditpol)    

**Credit Policy for Rate Negotiation**  

## Purpose  
This policy provides rules and guidance to staff when a customer contacts the bank seeking a rate negotiation. Staff must assess the customer’s eligibility based on key criteria to ensure fair and responsible lending practices.

## Eligibility Criteria  
Rate negotiations must consider the following factors:

### 1. Credit History  
- **Defaults & Arrears**: Customers with recent defaults or arrears (past 12 months) may not be eligible for rate reductions.
- **High-Risk Spending**: Frequent transactions at gambling institutions, payday lenders, or other high-risk entities may impact eligibility.

### 2. Borrower Type  
- **Age**: Customers nearing retirement (above 55) may require additional review based on future income stability.
- **Location**: Customers in high-risk postcodes (e.g., areas with declining property values) may not qualify for lower rates.
- **Occupation & Industry**: Customers employed in volatile industries (e.g.

In [ ]:
# # -- 6.0 LLM query, response and cleaning (format) --
# # Query (want to ingest credit policy, then see if it can determine relevant fields to pull back.
# # Then have it pull back those fields)

# #pre_prompt = sys + tbls
# data # this is from query above. Would want to incorporate that here to make a single call (or conver above to a func to call her)
# prompt0 = f"""You are tasked wtih assisting in the decision making process in relation to a customer rate negotiation.
#            We have access to the 
#            - credit policy {creditpol} which will be used to support or justify any lending decisions
#            - applicant data {data} which you can source any information you see fit to aid in the decision making process 
#            """
# question = "This cusomer want to borrow 3 millions dollars. Should we approve or reject?"
# prompt = question + prompt0
# response = chat_session.send_message(prompt)
# resp = response.text
# print(response.text)


In [ ]:
# working on the data (i.e. do i ask the questioj, share that with the credit policy, let it see the sql columns, then have it write the SQL?)

query0 = f""" Can you review the credit policy {creditpol} to determine what sort of information we might need
              You can then have look at the data you can access to help inform your response {tbls}
              Based on the above, you can then create a query (referring to {sys}) to extract the information you need
              As a reminder, you are only to produce the SQL code, not descriptioms, explanations, etc.
              You can only return a single query. If that means using CTE, subqueries, etc. then do so.
              """
#query = "Customer 1, would like to borrow a further 3 million dollars. How should we respond?"
query = "Customer 1, would like a .5 percent reduction on their Home Loan. How should we respond?"
prompt = query + query0 
response = chat_session.send_message(prompt)
resp = response.text
#print(response.text)

# Clean (format)
# Clean
resp1 = resp.replace('```sql','').replace('```','')
print(resp1)

In [ ]:
# -- 6.0 Query db, using LLM cleaned (format) llm response --
import sqlite3
from tabulate import tabulate
# Connect to the database
conn = sqlite3.connect("bank_data.db")
cursor = conn.cursor()

print('\n','-'*5, 'new', '-'*5)
# 4.0 Join query
query5 = f"""{resp1}"""
cursor.execute(query5)
rows = cursor.fetchall()
# Get column names
headers = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=headers, tablefmt="grid"))
data = tabulate(rows, headers=headers, tablefmt="grid")
conn.close()

In [ ]:
# -- 6.0 LLM query, response and cleaning (format) --
# Query (want to ingest credit policy, then see if it can determine relevant fields to pull back.
# Then have it pull back those fields)

#pre_prompt = sys + tbls
data # this is from query above. Would want to incorporate that here to make a single call (or conver above to a func to call her)
prompt0 = f"""You are tasked wtih assisting in the decision making process in relation to a customer rate negotiation.
           We have access to the 
           - credit policy {creditpol} which will be used to support or justify any lending decisions
           - applicant data {data} which you can source any information you see fit to aid in the decision making process.
           Based on the above, can you please an assessment on if the request is to be approved or rejected. 
           Format the response in Markdown, ensuring proper line breaks (\n) and spacing between sections
           """
question = query
prompt = question + prompt0
response = chat_session.send_message(prompt)
resp = response.text
#print(response.text)

#import textwrap
#long_text = response.text
#wrapped_text = textwrap.fill(long_text, width=120)  # Wrap at 40 characters
#print(wrapped_text)
from IPython.display import display, Markdown
display(Markdown(resp))


### Pilot set up for UI

#### UI set up v1

In [ ]:
# Adding key parts above to make functions for UI - 1st crack

In [ ]:
# -- 3.0 Set up LLM --
# ------------------------------------
# ---- Part i: llm -------------------
# ------------------------------------
import os
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import sqlite3


load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')
import os
import google.generativeai as genai
genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 0.2,
  "top_p": 0.90,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)
chat_session = model.start_chat(
  history=[  ])
#message = "this is a test only"
#response = chat_session.send_message(message)
#print(response.text)




# -- 4.0 System prompts --
# ------------------------------------
# ---- Part ii: hard code vals -------
# ------------------------------------
tbls = """There are 3 seperate tables, that are related by a common field 'customer_id'
         The tables are 
         Table 1: customers
         Columns: customer_id, name, age, location, tenure, occupation, industry_employed, income_monthly, credit_hist, defaults 
         Description: includes values such as income, occupation, credit history, defaults

         Table 2: products
         Columns: customer_id, product, description, rate, balance
         Description: includes values such as products, balances and rates

         Table 3: transactions
         Columns: customer_id, date, transaction_description, transaction_classification, trasaction_amount
         Description: transaction_classification column includes two values: deposits & withdrawals, 
                      transaction_description column includes many values, i.e. 
                      Credit Card Purchase - gambling, Deposit, Grocery Shopping, Salary Deposit, Utility Bill, Rent Payment, etc.
        """

sys = """You will create SQL code that can execute on a SQlite db.
         If request requires information contained in more that one table, 
         You have the ability to write subqueries, joins, etc. to get the data required.
         If the request requires aggregation, calculations, etc. you will include this in your query.
         Please pull back only fields, that you believe are relevant to the query.
         You will output only the code required to be execute on the SQLitedb.
         You will not include any descriptions, explainations, etc. - just output the code"""
         

with open('Credit Policy.txt', "r", encoding="utf-8") as f:
    creditpol = f.read()
# database_path = sqlite3.connect("bank_data.db")

# # Connect to SQLite (or create the database file)
# conn = sqlite3.connect("bank_data.db")
# database_path = conn.cursor()

# ------------------------------------
# ---- Part iii: code (create) -------
# ------------------------------------
# working on the data (i.e. do i ask the questioj, share that with the credit policy, let it see the sql columns, then have it write the SQL?)
def generate_sql_query(creditpol, tbls, sys, query):
    query0 = f"""Can you review the credit policy {creditpol} to determine what sort of information we might need?
                 You can then have a look at the data you can access to help inform your response {tbls}.
                 Based on the above, you can then create a query (referring to {sys}) to extract the information you need.
                 As a reminder, you are only to produce the SQL code, not descriptions, explanations, etc.
              """
    
    prompt = query + query0
    response = chat_session.send_message(prompt)
    resp = response.text

    # Clean (format)
    resp_cleaned = resp.replace('```sql', '').replace('```', '')
    return resp_cleaned

query = "Customer 1, would like a .5 percent reduction on their Home Loan. How should we respond?"
sql_query = generate_sql_query(creditpol, tbls, sys, query)
resp1 = sql_query
# print(resp1)

# ------------------------------------
# ---- Part iv: code (run) -----------
# ------------------------------------
# -- 6.0 Query db, using LLM cleaned (format) llm response --
# import sqlite3
# from tabulate import tabulate

# #def execute_sql_query(database_path, sql_query):
def execute_sql_query(resp1):
    conn = sqlite3.connect("bank_data.db")
    cursor = conn.cursor()

    query5 = f"""{resp1}"""
    cursor.execute(query5)
    rows = cursor.fetchall()
    # Get column names
    headers = [desc[0] for desc in cursor.description]
    #print(tabulate(rows, headers=headers, tablefmt="grid"))
    data = tabulate(rows, headers=headers, tablefmt="grid")
    conn.close()
    return data
    
print(execute_sql_query(resp1))
data = execute_sql_query(resp1)

# ------------------------------------
# ---- Part v: code (run) ------------
# ------------------------------------
# -- 6.0 LLM query, response and cleaning (format) --
# Query (want to ingest credit policy, then see if it can determine relevant fields to pull back.
# Then have it pull back those fields)

#pre_prompt = sys + tbls
#data # this is from query above. Would want to incorporate that here to make a single call (or conver above to a func to call her)
# from IPython.display import display, Markdown
# import gradio as gr
# def assess_rate_negotiation_ui(customer_number, request):
#     # data = execute_sql_query(sql_query)
#     prompt0 = f"""You are tasked with assisting in the decision-making process regarding a customer rate negotiation.
#                   We have access to:
#                   - Credit policy: {creditpol}, which will be used to support or justify any lending decisions.
#                   - Applicant data: {data}, from which you can source any relevant information to aid the decision-making process.
                  
#                   The customer identified as {customer_number} has made the following request:
#                   "{request}"

#                   Based on the above, please provide an assessment on whether the request should be approved or rejected.
#                   Format the response in Markdown, ensuring proper line breaks (\n) and spacing between sections.
#                """

#     response = chat_session.send_message(prompt0)
#     return response.text  # Return Markdown-formatted response

# customer_number = "customer 1"
# request = "would like a .5 percent reduction on their Home Loan. How should we respond?"
# print(assess_rate_negotiation_ui(customer_number, request))


# # Create Gradio UI
# # ------------------------------------
# # ---- Part vi: ui -------------------
# # ------------------------------------
# with gr.Blocks() as ui:
#     gr.Markdown("## Customer Rate Negotiation Assistant")
    
#     customer_number_input = gr.Textbox(label="Customer Number", placeholder="Enter Customer ID (e.g., 12345)")
#     request_input = gr.Textbox(label="Customer Request", placeholder="Enter the request (e.g., 0.5% rate reduction)")
    
#     output = gr.Markdown()  # Markdown-formatted output

#     submit_button = gr.Button("Assess Request")
#     submit_button.click(assess_rate_negotiation_ui, 
#                         inputs=[customer_number_input, request_input], 
#                         outputs=output)

# # Launch Gradio App
# ui.launch()



# query = "Customer 1, would like a .5 percent reduction on their Home Loan. How should we respond?"
# assess_rate_negotiation(creditpol, data, query)

#### UI set up v2

In [ ]:
# This is the baseline. Its good up to 2nd last function. New creating experiments of this (don't want loose this)
# Copy of above... trying to get it working, without loosing everything....


# -- 3.0 Set up LLM --
# ------------------------------------
# ---- Part i: llm -------------------
# ------------------------------------
import os
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import sqlite3


load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')
import os
import google.generativeai as genai
genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 0.2,
  "top_p": 0.90,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)
chat_session = model.start_chat(
  history=[  ])
#message = "this is a test only"
#response = chat_session.send_message(message)
#print(response.text)




# -- 4.0 System prompts --
# ------------------------------------
# ---- Part ii: hard code vals -------
# ------------------------------------
tbls = """There are 3 seperate tables, that are related by a common field 'customer_id'
         The tables are 
         Table 1: customers
         Columns: customer_id, name, age, location, tenure, occupation, industry_employed, income_monthly, credit_hist, defaults 
         Description: includes values such as income, occupation, credit history, defaults

         Table 2: products
         Columns: customer_id, product, description, rate, balance
         Description: includes values such as products, balances and rates

         Table 3: transactions
         Columns: customer_id, date, transaction_description, transaction_classification, trasaction_amount
         Description: transaction_classification column includes two values: deposits & withdrawals, 
                      transaction_description column includes many values, i.e. 
                      Credit Card Purchase - gambling, Deposit, Grocery Shopping, Salary Deposit, Utility Bill, Rent Payment, etc.
        """

sys = """You will create SQL code that can execute on a SQlite db.
         If request requires information contained in more that one table, 
         You have the ability to write subqueries, joins, etc. to get the data required.
         If the request requires aggregation, calculations, etc. you will include this in your query.
         Please pull back only fields, that you believe are relevant to the query.
         You will output only the code required to be execute on the SQLitedb.
         You will not include any descriptions, explainations, etc. - just output the code"""
         

with open('Credit Policy.txt', "r", encoding="utf-8") as f:
    creditpol = f.read()
# database_path = sqlite3.connect("bank_data.db")

# # Connect to SQLite (or create the database file)
# conn = sqlite3.connect("bank_data.db")
# database_path = conn.cursor()


# ------------------------------------
# ---- Part iv: code (run) -----------
# ------------------------------------
# -- 6.0 Query db, using LLM cleaned (format) llm response --
# import sqlite3
# from tabulate import tabulate

# #def execute_sql_query(database_path, sql_query):
def execute_sql_query(resp1):
    conn = sqlite3.connect("bank_data.db")
    cursor = conn.cursor()

    query5 = f"""{resp1}"""
    cursor.execute(query5)
    rows = cursor.fetchall()
    # Get column names
    headers = [desc[0] for desc in cursor.description]
    #print(tabulate(rows, headers=headers, tablefmt="grid"))
    data = tabulate(rows, headers=headers, tablefmt="grid")
    conn.close()
    return data
  

# ------------------------------------
# ---- Part iii: code (create) -------
# ------------------------------------
# working on the data (i.e. do i ask the questioj, share that with the credit policy, let it see the sql columns, then have it write the SQL?)
def generate_sql_query(creditpol, tbls, sys, query):
    query0 = f"""Can you review the credit policy {creditpol} to determine what sort of information we might need?
                 You can then have a look at the data you can access to help inform your response {tbls}.
                 Based on the above, you can then create a query (referring to {sys}) to extract the information you need.
                 As a reminder, you are only to produce the SQL code, not descriptions, explanations, etc.
              """
    
    prompt = query + query0
    response = chat_session.send_message(prompt)
    resp = response.text

    # Clean (format)
    resp_cleaned = resp.replace('```sql', '').replace('```', '')
    return resp_cleaned

# query = "Customer 1, would like a .5 percent reduction on their Home Loan. How should we respond?"
# sql_query = generate_sql_query(creditpol, tbls, sys, query)
resp1 = sql_query
#print(resp1)
#print(execute_sql_query(resp1))
data = execute_sql_query(resp1)


# Above good - 


def assess_rate_negotiation_ui(customer_number, query):
    # data = execute_sql_query(sql_query)
    prompt0 = f"""You are tasked with assisting in the decision-making process regarding a customer rate negotiation.
                  We have access to:
                  - Credit policy: {creditpol}, which will be used to support or justify any lending decisions.
                  - Applicant data: {data}, from which you can source any relevant information to aid the decision-making process.
                  
                  The customer identified as {customer_number} has made the following request:
                  "{query}"

                  Based on the above, please provide an assessment on whether the request should be approved or rejected.
                  Format the response in Markdown, ensuring proper line breaks (\n) and spacing between sections.
               """

    response = chat_session.send_message(prompt0)
    return response.text  # Return Markdown-formatted response

customer_number = "customer 1"
query = "would like a .5 percent reduction on their Home Loan. How should we respond?"
print(assess_rate_negotiation_ui(customer_number, request))


#### UI set up v3

In [ ]:
# experiment 1
# Copy of above... trying to get it working, without loosing everything....
# Can only try customers 1, 2 & 3 (there is not trans or prod data for others)
# Code works for all three, but sometimes it says it can't find cust. Run it again and it should work.
# this is now the BENCHMARK - do not edit (even though a bit dodgy as per above)
# Next add UI, then de-bug above issue.



# -- 3.0 Set up LLM --
# ------------------------------------
# ---- Part i: llm -------------------
# ------------------------------------
import os
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import sqlite3
import sqlite3
from tabulate import tabulate


load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')
import os
import google.generativeai as genai
genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 0.2,
  "top_p": 0.90,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)
chat_session = model.start_chat(
  history=[  ])
#message = "this is a test only"
#response = chat_session.send_message(message)
#print(response.text)




# -- 4.0 System prompts --
# ------------------------------------
# ---- Part ii: hard code vals -------
# ------------------------------------
tbls = """There are 3 seperate tables, that are related by a common field 'customer_id'
         The tables are 
         Table 1: customers
         Columns: customer_id, name, age, location, tenure, occupation, industry_employed, income_monthly, credit_hist, defaults 
         Description: includes values such as income, occupation, credit history, defaults

         Table 2: products
         Columns: customer_id, product, description, rate, balance
         Description: includes values such as products, balances and rates

         Table 3: transactions
         Columns: customer_id, date, transaction_description, transaction_classification, trasaction_amount
         Description: transaction_classification column includes two values: deposits & withdrawals, 
                      transaction_description column includes many values, i.e. 
                      Credit Card Purchase - gambling, Deposit, Grocery Shopping, Salary Deposit, Utility Bill, Rent Payment, etc.
        """

sys = """You will create SQL code that can execute on a SQlite db.
         If request requires information contained in more that one table, 
         You have the ability to write subqueries, joins, etc. to get the data required.
         If the request requires aggregation, calculations, etc. you will include this in your query.
         Please pull back only fields, that you believe are relevant to the query.
         You will output only the code required to be execute on the SQLitedb.
         You will not include any descriptions, explainations, etc. - just output the code"""
         

with open('Credit Policy.txt', "r", encoding="utf-8") as f:
    creditpol = f.read()
# database_path = sqlite3.connect("bank_data.db")

# # Connect to SQLite (or create the database file)
# conn = sqlite3.connect("bank_data.db")
# database_path = conn.cursor()


# ------------------------------------
# ---- Part iv: code (run) -----------
# ------------------------------------
# -- 6.0 Query db, using LLM cleaned (format) llm response --
# import sqlite3
# from tabulate import tabulate

# #def execute_sql_query(database_path, sql_query):
def execute_sql_query(resp1):
    conn = sqlite3.connect("bank_data.db")
    cursor = conn.cursor()

    query5 = f"""{resp1}"""
    cursor.execute(query5)
    rows = cursor.fetchall()
    # Get column names
    headers = [desc[0] for desc in cursor.description]
    #print(tabulate(rows, headers=headers, tablefmt="grid"))
    data = tabulate(rows, headers=headers, tablefmt="grid")
    conn.close()
    return data
  

# ------------------------------------
# ---- Part iii: code (create) -------
# ------------------------------------

def generate_and_assess_query(creditpol, tbls, sys, customer_number, query):


    # Step 1: Generate the SQL query
    query0 = f"""Can you review the credit policy {creditpol} to determine what sort of information we might need?
                 You can then have a look at the data you can access to help inform your response {tbls}.
                 Based on the above, you can then create a query (referring to {sys}) to extract the information you need.
                 As a reminder, you are only to produce the SQL code, not descriptions, explanations, etc.
              """
    prompt = query + query0
    response = chat_session.send_message(prompt)
    resp = response.text

    # Clean (format) the SQL response
    resp_cleaned = resp.replace('```sql', '').replace('```', '')

    # Step 2: Execute the SQL query
    data = execute_sql_query(resp_cleaned)

    # Step 3: Assess the rate negotiation for the customer
    prompt0 = f"""You are tasked with assisting in the decision-making process regarding a customer rate negotiation.
                  We have access to:
                  - Credit policy: {creditpol}, which will be used to support or justify any lending decisions.
                  - Applicant data: {data}, from which you can source any relevant information to aid the decision-making process.
                  
                  The customer identified as {customer_number} has made the following request:
                  "{query}"

                  Based on the above, please provide an assessment on whether the request should be approved or rejected.
                  Format the response in Markdown, ensuring proper line breaks (\n) and spacing between sections.
               """

    assessment_response = chat_session.send_message(prompt0)
    return assessment_response.text  # Return Markdown-formatted response




customer_number = "2"
query = "would like a .5 percent reduction on their Home Loan. How should we respond?"
print(generate_and_assess_query(creditpol, tbls, sys, customer_number, query))


#### UI set up v4

In [ ]:
# experiment 2
# Copy of experiment 1, but adding ui



# -- 3.0 Set up LLM --
# ------------------------------------
# ---- Part i: llm -------------------
# ------------------------------------
import os
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import sqlite3
import sqlite3
from tabulate import tabulate


load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')
import os
import google.generativeai as genai
genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 0.2,
  "top_p": 0.90,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)
chat_session = model.start_chat(
  history=[  ])
#message = "this is a test only"
#response = chat_session.send_message(message)
#print(response.text)




# -- 4.0 System prompts --
# ------------------------------------
# ---- Part ii: hard code vals -------
# ------------------------------------
tbls = """There are 3 seperate tables, that are related by a common field 'customer_id'
         The tables are 
         Table 1: customers
         Columns: customer_id, name, age, location, tenure, occupation, industry_employed, income_monthly, credit_hist, defaults 
         Description: includes values such as income, occupation, credit history, defaults

         Table 2: products
         Columns: customer_id, product, description, rate, balance
         Description: includes values such as products, balances and rates

         Table 3: transactions
         Columns: customer_id, date, transaction_description, transaction_classification, trasaction_amount
         Description: transaction_classification column includes two values: deposits & withdrawals, 
                      transaction_description column includes many values, i.e. 
                      Credit Card Purchase - gambling, Deposit, Grocery Shopping, Salary Deposit, Utility Bill, Rent Payment, etc.
        """

sys = """You will create SQL code that can execute on a SQlite db.
         If request requires information contained in more that one table, 
         You have the ability to write subqueries, joins, etc. to get the data required.
         If the request requires aggregation, calculations, etc. you will include this in your query.
         Please pull back only fields, that you believe are relevant to the query.
         You will output only the code required to be execute on the SQLitedb.
         You will not include any descriptions, explainations, etc. - just output the code"""
         

with open('Credit Policy.txt', "r", encoding="utf-8") as f:
    creditpol = f.read()
# database_path = sqlite3.connect("bank_data.db")

# # Connect to SQLite (or create the database file)
# conn = sqlite3.connect("bank_data.db")
# database_path = conn.cursor()


# ------------------------------------
# ---- Part iv: code (run) -----------
# ------------------------------------
# -- 6.0 Query db, using LLM cleaned (format) llm response --
# import sqlite3
# from tabulate import tabulate

# #def execute_sql_query(database_path, sql_query):
def execute_sql_query(resp1):
    conn = sqlite3.connect("bank_data.db")
    cursor = conn.cursor()

    query5 = f"""{resp1}"""
    cursor.execute(query5)
    rows = cursor.fetchall()
    # Get column names
    headers = [desc[0] for desc in cursor.description]
    #print(tabulate(rows, headers=headers, tablefmt="grid"))
    data = tabulate(rows, headers=headers, tablefmt="grid")
    conn.close()
    return data
  

# ------------------------------------
# ---- Part iii: code (create) -------
# ------------------------------------

def generate_and_assess_query(creditpol, tbls, sys, customer_id, query):


    # Step 1: Generate the SQL query
    query0 = f"""Can you review the credit policy {creditpol} to determine what sort of information we might need?
                 You can then have a look at the data you can access to help inform your response {tbls}.
                 Based on the above, you can then create a query (referring to {sys}) to extract the information you need.
                 As a reminder, you are only to produce the SQL code, not descriptions, explanations, etc.
                 You can only return a single query. If that means using CTE, subqueries, etc. then do so.
              """
    prompt = query + query0
    response = chat_session.send_message(prompt)
    resp = response.text

    # Clean (format) the SQL response
    resp_cleaned = resp.replace('```sql', '').replace('```', '')

    # Step 2: Execute the SQL query
    data = execute_sql_query(resp_cleaned)

    # Step 3: Assess the rate negotiation for the customer
    prompt0 = f"""You are tasked with assisting in the decision-making process regarding a customer rate negotiation.
                  We have access to:
                  - Credit policy: {creditpol}, which will be used to support or justify any lending decisions.
                  - Applicant data: {data}, from which you can source any relevant information to aid the decision-making process.
                  
                  The customer identified via the customer_id {customer_id} has made the following request:
                  "{query}"

                  Based on the above, please provide an assessment on whether the request should be approved or rejected.
                  Format the response in Markdown, ensuring proper line breaks (\n) and spacing between sections.
               """

    assessment_response = chat_session.send_message(prompt0)
    print(resp_cleaned)
    return assessment_response.text  # Return Markdown-formatted response




# customer_number = "2"
# query = "would like a .5 percent reduction on their Home Loan. How should we respond?"
# print(generate_and_assess_query(creditpol, tbls, sys, customer_number, query))


import gradio as gr

def query_function(customer_id, query):
    return generate_and_assess_query(creditpol, tbls, sys, customer_id, query)

# Predefined variables
creditpol = creditpol
tbls = tbls
sys = sys

demo = gr.Interface(
    fn=query_function,
    inputs=[
        gr.Textbox(label="customer id"),
        gr.Textbox(label="Query")
    ],
    #outputs=gr.Textbox(label="Response"),
    outputs=gr.Markdown(label="Response"),  # Changed to gr.Markdown
    title="Customer Query Assessment",
    description="Enter customer number and query to receive an assessment."
)

demo.launch()


# customer_number = "2"
# query = "would like a .5 percent reduction on their Home Loan. How should we respond?"
# print(generate_and_assess_query(creditpol, tbls, sys, customer_number, query))

#### UI set up v5

In [ ]:
# The above, works, but sometimes it returns the wrong or not customer.
# This is a copy of that and trying to resolve that issue
# Updated code and test thus far indicate issue resolved.

# All seems to be working.
# An next step (if moving to prod), could be to define a template for fixed output.



# -- 3.0 Set up LLM --
# ------------------------------------
# ---- Part i: llm -------------------
# ------------------------------------
import os
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import sqlite3
import sqlite3
from tabulate import tabulate


load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')
import os
import google.generativeai as genai
genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 0.2,
  "top_p": 0.90,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)
chat_session = model.start_chat(
  history=[  ])
#message = "this is a test only"
#response = chat_session.send_message(message)
#print(response.text)


#Instruction: query requires where customer_id = 

# -- 4.0 System prompts --
# ------------------------------------
# ---- Part ii: hard code vals -------
# ------------------------------------
tbls = """There are 3 seperate tables, that are related by a common field 'customer_id'
         The tables are 
         Table 1: customers
         Columns: customer_id, name, age, location, tenure, occupation, industry_employed, income_monthly, credit_hist, defaults 
         Description: includes values such as income, occupation, credit history, defaults
         
         Table 2: products
         Columns: customer_id, product, description, rate, balance
         Description: includes values such as products, balances and rates

         Table 3: transactions
         Columns: customer_id, date, transaction_description, transaction_classification, trasaction_amount
         Description: transaction_classification column includes two values: deposits & withdrawals, 
                      transaction_description column includes many values, i.e. 
                      Credit Card Purchase - gambling, Deposit, Grocery Shopping, Salary Deposit, Utility Bill, Rent Payment, etc.
         
                
        """

sys = """You will create SQL code that can execute on a SQlite db.
         If request requires information contained in more that one table, 
         You have the ability to write subqueries, joins, etc. to get the data required.
         If the request requires aggregation, calculations, etc. you will include this in your query.
         Please pull back only fields, that you believe are relevant to the query.
         You will output only the code required to be execute on the SQLitedb.
         You will not include any descriptions, explainations, etc. - just output the code"""
         

with open('Data/Credit Policy.txt', "r", encoding="utf-8") as f:
    creditpol = f.read()
# database_path = sqlite3.connect("bank_data.db")

# # Connect to SQLite (or create the database file)
# conn = sqlite3.connect("bank_data.db")
# database_path = conn.cursor()


# ------------------------------------
# ---- Part iii: code (run) -----------
# ------------------------------------
# -- 6.0 Query db, using LLM cleaned (format) llm response --
# import sqlite3
# from tabulate import tabulate

# #def execute_sql_query(database_path, sql_query):
def execute_sql_query(resp1):
    conn = sqlite3.connect("bank_data.db")
    cursor = conn.cursor()

    query5 = f"""{resp1}"""
    cursor.execute(query5)
    rows = cursor.fetchall()
    # Get column names
    headers = [desc[0] for desc in cursor.description]
    #print(tabulate(rows, headers=headers, tablefmt="grid"))
    data = tabulate(rows, headers=headers, tablefmt="grid")
    conn.close()
    return data
  

# ------------------------------------
# ---- Part iv: code (create) -------
# ------------------------------------

def generate_and_assess_query(creditpol, tbls, sys, customer_id, query):

    # Step 1: Generate the SQL query
    query0 = f"""Can you review the credit policy {creditpol} to determine what sort of information we might need?
                 You can then have a look at the data you can access to help inform your response {tbls}.
                 Based on the above, 
                 - you can then create a query (referring to {sys}) to extract the information you need.
                 - filtering query to the target customer {customer_id}

                 As a reminder, you are only to produce the SQL code, not descriptions, explanations, etc.
                 You can only return a single query. If that means using CTE, subqueries, etc. then do so.
                                  
              """
    prompt = query + query0
    response = chat_session.send_message(prompt)
    resp = response.text

    # Clean (format) the SQL response
    resp_cleaned = resp.replace('```sql', '').replace('```', '')

    # Step 2: Execute the SQL query
    data = execute_sql_query(resp_cleaned)

    # Step 3: Assess the rate negotiation for the customer
    prompt0 = f"""You are tasked with assisting in the decision-making process regarding a customer rate negotiation.
                  We have access to:
                  - Credit policy: {creditpol}, which will be used to support or justify any lending decisions.
                  - Applicant data: {data}, from which you can source any relevant information to aid the decision-making process.
                  
                  The customer identified via the customer_id {customer_id} has made the following request:
                  "{query}"

                  Based on the above, please provide an assessment on whether the request should be approved or rejected.
                  Format the response in Markdown, ensuring proper line breaks (\n) and spacing between sections.
               """

    assessment_response = chat_session.send_message(prompt0)
    print(resp_cleaned)
    return assessment_response.text  # Return Markdown-formatted response



# customer_number = "2"
# query = "would like a .5 percent reduction on their Home Loan. How should we respond?"
# print(generate_and_assess_query(creditpol, tbls, sys, customer_number, query))


# ------------------------------------
# ---- Part V: ui --------------------
# ------------------------------------
import gradio as gr
def query_function(customer_id, query):
    return generate_and_assess_query(creditpol, tbls, sys, customer_id, query)

# Predefined variables
creditpol = creditpol
tbls = tbls
sys = sys

demo = gr.Interface(
    fn=query_function,
    inputs=[
        gr.Textbox(label="customer id"),
        gr.Textbox(label="Query")
    ],
    #outputs=gr.Textbox(label="Response"),
    outputs=gr.Markdown(label="Response"),  # Changed to gr.Markdown
    title="Customer Query Assessment",
    description="Enter customer number and query to receive an assessment."
)

demo.launch()


# customer_id = 3
# query = "would like a .5 percent reduction on their Home Loan. How should we respond?"
# print(generate_and_assess_query(creditpol, tbls, sys, customer_id, query))

  # (1, "Alice Johnson", 34, "Sydney", 10, "Stock broker", "Finance", 10000, "Fair", 0),
  # (2, "Bob Smith", 45, "Melbourne", 15, "Waiter/Waitress", "Accommodation", 500, "Poor", 2),
  # (3, "Charlie Brown", 29, "Brisbane", 5, "Architect", "Construction", 8000, "Good", 0), 

* Running on local URL:  http://127.0.0.1:7890

To create a public link, set `share=True` in `launch()`.



SELECT
  c.customer_id,
  c.name,
  c.age,
  c.location,
  c.tenure,
  c.occupation,
  c.industry_employed,
  c.income_monthly,
  c.credit_hist,
  c.defaults,
  p.product,
  p.balance,
  p.rate,
  SUM(CASE WHEN t.transaction_classification = 'withdrawal' THEN t.trasaction_amount ELSE 0 END) AS total_withdrawals,
  SUM(CASE WHEN t.transaction_description LIKE '%gambling%' THEN t.trasaction_amount ELSE 0 END) AS gambling_transactions
FROM customers AS c
INNER JOIN products AS p
  ON c.customer_id = p.customer_id
LEFT JOIN transactions AS t
  ON c.customer_id = t.customer_id
WHERE
  c.customer_id = 3
GROUP BY
  c.customer_id,
  c.name,
  c.age,
  c.location,
  c.tenure,
  c.occupation,
  c.industry_employed,
  c.income_monthly,
  c.credit_hist,
  c.defaults,
  p.product,
  p.balance,
  p.rate;



SELECT
  c.customer_id,
  c.name,
  c.age,
  c.location,
  c.tenure,
  c.occupation,
  c.industry_employed,
  c.income_monthly,
  c.credit_hist,
  c.defaults,
  p.product,
  p.balance,
  p.rate,